# Dynamic Programming
###### (Algorithm Design with Haskell)

Optimization problems that can be expressed recursively can often be solved using dynamic programming techniques. If the recursion happens inductively then a thinning algorithm may be preferrable.

One such problem is the need to find the least expensive sequence of operations needed to transform one string into another.

Here is a recursive definition

In [12]:
:m Data.List
:m Data.Ord

data Op
  = Copy Char
  | Replace Char Char
  | Insert Char
  | Delete Char
  deriving (Show)
  
ecost :: Op -> Int
ecost Copy{} = 0
ecost Replace{} = 3
ecost Insert{} = 2
ecost Delete{} = 2

cost :: [Op] -> Int
cost = sum . map ecost

minWith :: Ord a => (x -> a) -> [x] -> x
minWith f = snd . minimumBy (comparing fst) . map (\x -> (f x, x))

mce :: String -> String -> [Op]
mce xs = head . foldr nextStep firstStep where
  firstStep = tails $ Delete <$> xs
  nextStep y row = foldr add [Insert y : head row] es where
    es = zip3 row (tail row) xs
    add (es1, es2, x) r
      | x == y = (Copy x : es2) : r
      | otherwise =
          minWith cost
          [ Insert y : es1
          , Delete x : head r
          , Replace x y : es2
          ] : r
          
mce "abca" "bac"

[Insert 'b',Copy 'a',Delete 'b',Copy 'c',Delete 'a']

The key idea is that we build the dependency graph of the recursion row-wise from bottom to top and each row from right to left.

Different operations depend on different previous operations, in the current or previous row.

This can be improved by caching the cost instead of calculating it from scratch each time.

Consider the shuttle-bus problem where a bus picks up some number of passengers who each have a destination along the buses route. The bus is only able to make at most `k` stops so the stops must be optimized to minimize the cost to the passengers which is the distance from the stop that they wanted to get off at.

In [24]:
type Stop = Int
type Passengers = [(Int, Stop)]

pickStop :: [(Int, Stop)] -> (Int, Stop)
pickStop ps = minimum $ do
  (v, (_, s):u) <- zip (inits ps) (tails ps)
  let c = sum $ (\(n,st) -> n * abs (st - s)) <$> v ++ u
  pure (c, s)

-- naive recursion. numStops should be 1 less than required.
-- This doesn't try stops that are between stops that people are going to. Does this matter?
shuttle :: Passengers -> Int -> (Int, [Int])
shuttle [] _ = (0, [])
shuttle passengers 0 = let (c, cheapest) = pickStop passengers in (c, [cheapest])
shuttle passengers numStops = minimumBy (comparing fst) go where
  pairs = tail $ zip (inits passengers) (tails passengers)
  go = do
    (off, on) <- pairs
    let (c, cheapest) = pickStop off
        (tc, rest) = shuttle on (numStops - 1)
    pure $ (c + tc, cheapest : rest)
    
shuttle [(3,1), (10,2), (5,3), (15,4), (4,5), (10,8), (22,10)] 4

(7,[2,3,4,8,10])

Here's the algorithm from the book.

In [33]:
type Leg = (Int, Int)

apply :: Int -> (a -> a) -> a -> a
apply 0 _ x = x
apply n f x = f (apply (n - 1) f x)

cost :: Passengers -> [Leg] -> Int
cost _ [] = 0
cost ps ((x,y):ls) = legCost qs (x, y) + cost rs ls where
  (qs, rs) = span ((<= y) . snd) ps
  legCost ps (x, y) = sum [c * min (s - x) (y - s) | (c, s) <- ps]

schedule :: Int -> Int -> Passengers -> [Leg]
schedule n k ps = head (apply k step start) where
  start = [[(x,n)] | x <- [0..n-1]] ++ [[]] -- all possible Legs starting at 0 plus the empty list
  step t = zipWith entry [0..n-1] (tail $ tails t) ++ [[]]
  entry x ts = minWith (cost (cut x ps)) -- only select on list of Legs
                 (zipWith (:) [(x,y) | y <- [x+1..n]] ts) -- this is picking all Legs starting at x
  cut x = dropWhile ((<= x) . snd)
  
schedule 10 4 [(3,1), (10,2), (5,3), (15,4), (4,5), (10,8), (22,10)]

[(0,2),(2,3),(3,4),(4,8),(8,10)]